In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import optuna

# 导入训练集和测试集
train_data = pd.read_excel(r'C:\Users\k\Desktop\硕士论文\ORP\ORP4\XGBoost\ORP_function_train.xlsx')
test_data = pd.read_excel(r'C:\Users\k\Desktop\硕士论文\ORP\ORP4\XGBoost\ORP_function_test.xlsx')

# 分离特征和目标变量
X_train, y_train = train_data.drop(['E','File'], axis=1), train_data['E']
X_test, y_test = test_data.drop(['E','File'], axis=1), test_data['E']

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'gamma': trial.suggest_float('gamma', 0, 0.6),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0)
    }

    model = XGBRegressor(**param)
    
    # 使用k折交叉验证
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold)
        y_val_pred = model.predict(X_val_fold)
        rmse_fold = np.sqrt(mean_squared_error(y_val_fold, y_val_pred))
        rmse_scores.append(rmse_fold)
    
    # 计算交叉验证的平均RMSE
    avg_rmse = np.mean(rmse_scores)
    
    return avg_rmse

# 创建Optuna试验
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

# 获取最佳超参数
best_params = study.best_params
print("最佳超参数组合：", best_params)

# 使用最佳超参数训练模型  
best_xgb_reg = XGBRegressor(**best_params)
best_xgb_reg.fit(X_train, y_train)  

# 预测训练集和测试集数据  
y_train_pred = best_xgb_reg.predict(X_train)  
y_test_pred = best_xgb_reg.predict(X_test)  

# 计算训练集和测试集的均方根误差 (RMSE) 和决定系数 (R2)  
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))  
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))  

train_r2 = r2_score(y_train, y_train_pred)  
test_r2 = r2_score(y_test, y_test_pred)  

# 输出 RMSE 和 R2  
print("训练集的均方根误差 (RMSE)：", train_rmse)  
print("测试集的均方根误差 (RMSE)：", test_rmse)  
print("训练集的决定系数 (R2)：", train_r2)  
print("测试集的决定系数 (R2)：", test_r2)


[I 2024-02-22 18:31:25,699] A new study created in memory with name: no-name-11f01b2b-1200-421b-8b98-13f2dba6b00f
[I 2024-02-22 18:31:27,001] Trial 0 finished with value: 0.46014135360357483 and parameters: {'learning_rate': 0.8141077010607489, 'n_estimators': 391, 'max_depth': 2, 'min_child_weight': 2, 'gamma': 0.45625239538687096, 'subsample': 0.903989463885902, 'colsample_bytree': 0.87502420088893}. Best is trial 0 with value: 0.46014135360357483.
[I 2024-02-22 18:31:27,448] Trial 1 finished with value: 0.41759161284131274 and parameters: {'learning_rate': 0.27080933261422846, 'n_estimators': 473, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.16193696044019149, 'subsample': 0.8396972452417255, 'colsample_bytree': 0.9419319694270774}. Best is trial 1 with value: 0.41759161284131274.
[I 2024-02-22 18:31:27,865] Trial 2 finished with value: 0.4858664160965686 and parameters: {'learning_rate': 0.9825202107339555, 'n_estimators': 451, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0

[I 2024-02-22 18:31:36,899] Trial 24 finished with value: 0.42466767144739065 and parameters: {'learning_rate': 0.07998604273973736, 'n_estimators': 362, 'max_depth': 3, 'min_child_weight': 4, 'gamma': 0.23107732317406218, 'subsample': 0.6999570947394402, 'colsample_bytree': 0.9158843336621302}. Best is trial 16 with value: 0.4173500707688283.
[I 2024-02-22 18:31:37,097] Trial 25 finished with value: 0.43006139553420775 and parameters: {'learning_rate': 0.1009790047847297, 'n_estimators': 109, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0.05791283591814622, 'subsample': 0.8186088967128756, 'colsample_bytree': 0.7754746474236761}. Best is trial 16 with value: 0.4173500707688283.
[I 2024-02-22 18:31:37,458] Trial 26 finished with value: 0.4367329796408154 and parameters: {'learning_rate': 0.1750715973414686, 'n_estimators': 311, 'max_depth': 5, 'min_child_weight': 4, 'gamma': 0.31981209622646356, 'subsample': 0.8920266478515902, 'colsample_bytree': 0.9703116107312344}. Best is trial 

[I 2024-02-22 18:31:46,006] Trial 48 finished with value: 0.43762180064952616 and parameters: {'learning_rate': 0.13963806326397582, 'n_estimators': 458, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0.127403057264895, 'subsample': 0.9192992303972328, 'colsample_bytree': 0.9219043904045489}. Best is trial 16 with value: 0.4173500707688283.
[I 2024-02-22 18:31:46,544] Trial 49 finished with value: 0.5267205093623731 and parameters: {'learning_rate': 0.9136207728750863, 'n_estimators': 428, 'max_depth': 3, 'min_child_weight': 3, 'gamma': 0.004681495506712749, 'subsample': 0.6832773104953238, 'colsample_bytree': 0.9497893433916988}. Best is trial 16 with value: 0.4173500707688283.
[I 2024-02-22 18:31:46,797] Trial 50 finished with value: 0.42702520781779246 and parameters: {'learning_rate': 0.31593869236955474, 'n_estimators': 193, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0.10001727849283557, 'subsample': 0.8038435459629129, 'colsample_bytree': 0.9734819853738113}. Best is trial 

[I 2024-02-22 18:31:56,930] Trial 72 finished with value: 0.41841134478814535 and parameters: {'learning_rate': 0.06995829183889994, 'n_estimators': 343, 'max_depth': 5, 'min_child_weight': 5, 'gamma': 0.01767016797662965, 'subsample': 0.6005974829181173, 'colsample_bytree': 0.7156932962370129}. Best is trial 63 with value: 0.40813214828344185.
[I 2024-02-22 18:31:57,295] Trial 73 finished with value: 0.4168676483413171 and parameters: {'learning_rate': 0.16374710452717697, 'n_estimators': 307, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0.10787808403554147, 'subsample': 0.6473132349981818, 'colsample_bytree': 0.6472962547096457}. Best is trial 63 with value: 0.40813214828344185.
[I 2024-02-22 18:31:57,710] Trial 74 finished with value: 0.4163454659195024 and parameters: {'learning_rate': 0.1076799664060675, 'n_estimators': 324, 'max_depth': 5, 'min_child_weight': 5, 'gamma': 0.049221820695989474, 'subsample': 0.6732649572048373, 'colsample_bytree': 0.6439270711888534}. Best is tri

[I 2024-02-22 18:32:06,619] Trial 96 finished with value: 0.41056340830605853 and parameters: {'learning_rate': 0.05501822421203909, 'n_estimators': 265, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0.10302244539361063, 'subsample': 0.6077543723968806, 'colsample_bytree': 0.7182103411389458}. Best is trial 94 with value: 0.40518856220776794.
[I 2024-02-22 18:32:06,978] Trial 97 finished with value: 0.4083218954944222 and parameters: {'learning_rate': 0.05833226859026107, 'n_estimators': 267, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.09936074669814676, 'subsample': 0.6129866463302057, 'colsample_bytree': 0.6596427568676249}. Best is trial 94 with value: 0.40518856220776794.
[I 2024-02-22 18:32:07,343] Trial 98 finished with value: 0.40928537478523064 and parameters: {'learning_rate': 0.06023945279632177, 'n_estimators': 268, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.0965326259787252, 'subsample': 0.6089701150622288, 'colsample_bytree': 0.655911975770566}. Best is tria

...

[I 2024-02-22 18:37:11,312] Trial 912 finished with value: 0.4180413086378626 and parameters: {'learning_rate': 0.10422250984569303, 'n_estimators': 200, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.05441807306766251, 'subsample': 0.6083936232035446, 'colsample_bytree': 0.6924740668426705}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:11,706] Trial 913 finished with value: 0.40822304734993764 and parameters: {'learning_rate': 0.03646681134082578, 'n_estimators': 246, 'max_depth': 3, 'min_child_weight': 3, 'gamma': 0.07812120910147172, 'subsample': 0.6621334843846413, 'colsample_bytree': 0.6698453487560594}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:12,112] Trial 914 finished with value: 0.4115585901870095 and parameters: {'learning_rate': 0.06991949346872418, 'n_estimators': 232, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.03579635766396645, 'subsample': 0.621382068056306, 'colsample_bytree': 0.6590027222906295}. Best is t

[I 2024-02-22 18:37:21,076] Trial 936 finished with value: 0.4146548845990946 and parameters: {'learning_rate': 0.03799644425460955, 'n_estimators': 180, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.06856361435756657, 'subsample': 0.6158876337720448, 'colsample_bytree': 0.6802701028669416}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:21,453] Trial 937 finished with value: 0.4401998069103431 and parameters: {'learning_rate': 0.01632478435124674, 'n_estimators': 196, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.058716582052324565, 'subsample': 0.6300181657746411, 'colsample_bytree': 0.7007049953846041}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:21,945] Trial 938 finished with value: 0.4293910844203054 and parameters: {'learning_rate': 0.08009823658803691, 'n_estimators': 287, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.00011325799029705525, 'subsample': 0.7720920386353759, 'colsample_bytree': 0.6899118192537331}. Best 

[I 2024-02-22 18:37:30,981] Trial 960 finished with value: 0.4424652711142322 and parameters: {'learning_rate': 0.11241214150519159, 'n_estimators': 295, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.508994069444976, 'subsample': 0.7487527218542244, 'colsample_bytree': 0.664782624900378}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:31,378] Trial 961 finished with value: 0.407374978560061 and parameters: {'learning_rate': 0.059851144227304705, 'n_estimators': 250, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.10004807561517681, 'subsample': 0.6398693883631227, 'colsample_bytree': 0.7178528550259752}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:31,797] Trial 962 finished with value: 0.4279896117773969 and parameters: {'learning_rate': 0.01724240913515284, 'n_estimators': 231, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.046294521700755, 'subsample': 0.6306611437370313, 'colsample_bytree': 0.6339112794628976}. Best is trial 

[I 2024-02-22 18:37:41,059] Trial 984 finished with value: 0.6567496004813164 and parameters: {'learning_rate': 0.002521567279309536, 'n_estimators': 201, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.580367853159894, 'subsample': 0.6080028921948842, 'colsample_bytree': 0.713822944947896}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:41,466] Trial 985 finished with value: 0.41597528915998794 and parameters: {'learning_rate': 0.06383956957502006, 'n_estimators': 254, 'max_depth': 3, 'min_child_weight': 3, 'gamma': 0.04631679438543228, 'subsample': 0.6609364292481389, 'colsample_bytree': 0.6942368037356693}. Best is trial 756 with value: 0.3991852011247386.
[I 2024-02-22 18:37:41,875] Trial 986 finished with value: 0.41716231764233136 and parameters: {'learning_rate': 0.1113816077882542, 'n_estimators': 280, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.07312008798748772, 'subsample': 0.6478308923317739, 'colsample_bytree': 0.6602892847926802}. Best is tr

最佳超参数组合： {'learning_rate': 0.048765542112345836, 'n_estimators': 235, 'max_depth': 4, 'min_child_weight': 3, 'gamma': 0.03675065468295996, 'subsample': 0.6071639846574391, 'colsample_bytree': 0.6845145522272096}
训练集的均方根误差 (RMSE)： 0.2110674190607092
测试集的均方根误差 (RMSE)： 0.4453547773897717
训练集的决定系数 (R2)： 0.9321181543342589
测试集的决定系数 (R2)： 0.7071573366057609


In [2]:
from sklearn.inspection import permutation_importance
# 使用最佳超参数训练模型  
best_xgb_reg.fit(X_train, y_train)  

# 计算训练集上的Permutation Importance
perm_importance = permutation_importance(best_xgb_reg, X_train, y_train, n_repeats=30, random_state=42)

# 获取特征名称
feature_names = X_train.columns

# 输出每个特征的重要性
for i in perm_importance.importances_mean.argsort()[::-1]:
     if perm_importance.importances_mean[i] - 2 * perm_importance.importances_std[i] > 0:
         print(f"{feature_names[i]:<8}"
               f"{perm_importance.importances_mean[i]:.3f}"
               f" +/- {perm_importance.importances_std[i]:.3f}")


A7      0.497 +/- 0.045
A30     0.244 +/- 0.030
A1      0.143 +/- 0.024
A3      0.080 +/- 0.016
A2      0.056 +/- 0.009
A9      0.054 +/- 0.010
A23     0.053 +/- 0.009
A13     0.033 +/- 0.006
A16     0.032 +/- 0.006
A6      0.027 +/- 0.005
A4      0.025 +/- 0.004
A18     0.024 +/- 0.006
A10     0.017 +/- 0.004
A28     0.015 +/- 0.002
A8      0.013 +/- 0.003
A22     0.010 +/- 0.002
A15     0.009 +/- 0.003
A19     0.008 +/- 0.003
A14     0.007 +/- 0.002
A17     0.004 +/- 0.002
A12     0.004 +/- 0.002
A11     0.004 +/- 0.001


In [6]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import optuna

# 导入训练集和测试集
train_data = pd.read_excel(r'C:\Users\k\Desktop\硕士论文\ORP\ORP4\XGBoost\ORP_function_train.xlsx')
test_data = pd.read_excel(r'C:\Users\k\Desktop\硕士论文\ORP\ORP4\XGBoost\ORP_function_test.xlsx')

# 分离特征和目标变量
X_train, y_train = train_data.drop(['E','File'], axis=1), train_data['E']
X_test, y_test = test_data.drop(['E','File'], axis=1), test_data['E']

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'gamma': trial.suggest_float('gamma', 0, 0.6),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0)
    }

    model = XGBRegressor(**param)
    
    # 使用k折交叉验证
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_index, val_index in kf.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold)
        y_val_pred = model.predict(X_val_fold)
        rmse_fold = np.sqrt(mean_squared_error(y_val_fold, y_val_pred))
        rmse_scores.append(rmse_fold)
    
    # 计算交叉验证的平均RMSE
    avg_rmse = np.mean(rmse_scores)
    
    return avg_rmse

# 创建Optuna试验
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

# 获取最佳超参数
best_params = study.best_params
print("最佳超参数组合：", best_params)

# 使用最佳超参数训练模型  
best_xgb_reg = XGBRegressor(**best_params)
# 使用最佳超参数训练模型
best_xgb_reg.fit(X_train, y_train)

# 计算训练集上的Permutation Importance
perm_importance = permutation_importance(best_xgb_reg, X_train, y_train, n_repeats=30, random_state=42)

# 获取特征名称
feature_names = X_train.columns

# 筛选重要性得分高于0.05的特征
important_features = [feature_names[i] for i in perm_importance.importances_mean.argsort()[::-1] if perm_importance.importances_mean[i] > 0.01]

# 输出选择的重要特征
print("选择的重要特征:", important_features)

# 使用筛选后的特征重新进行超参数优化
X_train_selected = X_train[important_features]
X_test_selected = X_test[important_features]

def objective(trial):
    # 在这里更新超参数空间
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'gamma': trial.suggest_float('gamma', 0, 0.6),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0)
    }
    model = XGBRegressor(**param)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []
    for train_index, val_index in kf.split(X_train_selected):
        X_train_fold, X_val_fold = X_train_selected.iloc[train_index], X_train_selected.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        model.fit(X_train_fold, y_train_fold)
        y_val_pred = model.predict(X_val_fold)
        rmse_fold = np.sqrt(mean_squared_error(y_val_fold, y_val_pred))
        rmse_scores.append(rmse_fold)
    avg_rmse = np.mean(rmse_scores)
    return avg_rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 获取新的最佳超参数
best_params = study.best_params
print("最佳超参数组合：", best_params)

# 使用新的最佳超参数和筛选后的特征训练模型
best_xgb_reg = XGBRegressor(**best_params)
best_xgb_reg.fit(X_train_selected, y_train)

# 预测训练集和测试集数据
y_train_pred = best_xgb_reg.predict(X_train_selected)
y_test_pred = best_xgb_reg.predict(X_test_selected)

# 计算训练集和测试集的均方根误差 (RMSE) 和决定系数 (R2)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# 输出 RMSE 和 R2
print("训练集的均方根误差 (RMSE)：", train_rmse)
print("测试集的均方根误差 (RMSE)：", test_rmse)
print("训练集的决定系数 (R2)：", train_r2)
print("测试集的决定系数 (R2)：", test_r2)

[I 2024-02-22 19:15:10,290] A new study created in memory with name: no-name-f00b41d4-97da-48bb-a239-2fc9ece48a9c
[I 2024-02-22 19:15:10,660] Trial 0 finished with value: 0.4382442071730849 and parameters: {'learning_rate': 0.39999086894480484, 'n_estimators': 365, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0.5937620163945592, 'subsample': 0.8275931192677999, 'colsample_bytree': 0.9307945278953633}. Best is trial 0 with value: 0.4382442071730849.
[I 2024-02-22 19:15:11,066] Trial 1 finished with value: 0.4614863406230347 and parameters: {'learning_rate': 0.576589000525306, 'n_estimators': 424, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.45154669632312827, 'subsample': 0.6740532316031953, 'colsample_bytree': 0.958797737990708}. Best is trial 0 with value: 0.4382442071730849.
[I 2024-02-22 19:15:11,343] Trial 2 finished with value: 0.4218984173147719 and parameters: {'learning_rate': 0.2552611408962762, 'n_estimators': 271, 'max_depth': 2, 'min_child_weight': 4, 'gamma': 0.194

[I 2024-02-22 19:15:18,003] Trial 24 finished with value: 0.4205160854622173 and parameters: {'learning_rate': 0.13022411672150366, 'n_estimators': 138, 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0.17344991039802943, 'subsample': 0.863644187997915, 'colsample_bytree': 0.9114929057376817}. Best is trial 24 with value: 0.4205160854622173.
[I 2024-02-22 19:15:18,206] Trial 25 finished with value: 0.42932476786450435 and parameters: {'learning_rate': 0.15295872723099777, 'n_estimators': 141, 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0.3016156943043047, 'subsample': 0.7866865319401016, 'colsample_bytree': 0.917892746755513}. Best is trial 24 with value: 0.4205160854622173.
[I 2024-02-22 19:15:18,415] Trial 26 finished with value: 0.43804488826805077 and parameters: {'learning_rate': 0.3065603279967095, 'n_estimators': 148, 'max_depth': 2, 'min_child_weight': 1, 'gamma': 0.2253668357143177, 'subsample': 0.8380240221178042, 'colsample_bytree': 0.8143650041411844}. Best is trial 24 

[I 2024-02-22 19:15:26,932] Trial 48 finished with value: 0.43263116969200716 and parameters: {'learning_rate': 0.25276244326782427, 'n_estimators': 342, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.1404286164301554, 'subsample': 0.8074974313169969, 'colsample_bytree': 0.609295616919522}. Best is trial 42 with value: 0.4013225513527641.
[I 2024-02-22 19:15:27,290] Trial 49 finished with value: 0.4410037549368293 and parameters: {'learning_rate': 0.4096880450117022, 'n_estimators': 314, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.06693100088634034, 'subsample': 0.7757692647906472, 'colsample_bytree': 0.892095621050125}. Best is trial 42 with value: 0.4013225513527641.
[I 2024-02-22 19:15:27,889] Trial 50 finished with value: 0.4366980638743937 and parameters: {'learning_rate': 0.21751277320409737, 'n_estimators': 409, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.0012687160850031742, 'subsample': 0.7991346932634456, 'colsample_bytree': 0.952761149756417}. Best is trial 42

[I 2024-02-22 19:15:37,260] Trial 72 finished with value: 0.413837049690685 and parameters: {'learning_rate': 0.21773586752289373, 'n_estimators': 410, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.016396504028407838, 'subsample': 0.8308809840384407, 'colsample_bytree': 0.831382909850142}. Best is trial 42 with value: 0.4013225513527641.
[I 2024-02-22 19:15:37,692] Trial 73 finished with value: 0.4029155357401297 and parameters: {'learning_rate': 0.16096149523871361, 'n_estimators': 388, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.08822233169432707, 'subsample': 0.8270375324856032, 'colsample_bytree': 0.8251450703899175}. Best is trial 42 with value: 0.4013225513527641.
[I 2024-02-22 19:15:38,162] Trial 74 finished with value: 0.4102056112093832 and parameters: {'learning_rate': 0.168532717267465, 'n_estimators': 395, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.019463998949790798, 'subsample': 0.8276961735247065, 'colsample_bytree': 0.8300648810743426}. Best is trial 42

[I 2024-02-22 19:15:47,909] Trial 96 finished with value: 0.4213697914655593 and parameters: {'learning_rate': 0.13946990526343234, 'n_estimators': 377, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.0924566312199422, 'subsample': 0.8091864594363015, 'colsample_bytree': 0.8697854899305477}. Best is trial 42 with value: 0.4013225513527641.
[I 2024-02-22 19:15:48,357] Trial 97 finished with value: 0.42350521109247297 and parameters: {'learning_rate': 0.27660153242643576, 'n_estimators': 426, 'max_depth': 5, 'min_child_weight': 2, 'gamma': 0.0745867466656308, 'subsample': 0.8841075461872252, 'colsample_bytree': 0.879192582349598}. Best is trial 42 with value: 0.4013225513527641.
[I 2024-02-22 19:15:48,812] Trial 98 finished with value: 0.4240451704515585 and parameters: {'learning_rate': 0.07226005935387086, 'n_estimators': 390, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.08353792714334346, 'subsample': 0.7821761550024979, 'colsample_bytree': 0.7800979791088353}. Best is trial 42

...

[I 2024-02-22 19:22:26,609] Trial 24 finished with value: 0.43590736853296636 and parameters: {'learning_rate': 0.19345178203673435, 'n_estimators': 454, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.4341810622407762, 'subsample': 0.7946674327822117, 'colsample_bytree': 0.6085177122068752}. Best is trial 2 with value: 0.41783690726821804.
[I 2024-02-22 19:22:27,010] Trial 25 finished with value: 0.40985038660366796 and parameters: {'learning_rate': 0.0664964875891969, 'n_estimators': 477, 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0.154283244422178, 'subsample': 0.7257649795690446, 'colsample_bytree': 0.7090883335741729}. Best is trial 25 with value: 0.40985038660366796.
[I 2024-02-22 19:22:27,407] Trial 26 finished with value: 0.42333063575783064 and parameters: {'learning_rate': 0.05290499989199224, 'n_estimators': 469, 'max_depth': 2, 'min_child_weight': 2, 'gamma': 0.14571584836481122, 'subsample': 0.7197544301375538, 'colsample_bytree': 0.7589648684881295}. Best is trial 

[I 2024-02-22 19:22:35,141] Trial 48 finished with value: 0.418163559463194 and parameters: {'learning_rate': 0.029123223215250918, 'n_estimators': 382, 'max_depth': 4, 'min_child_weight': 1, 'gamma': 0.1618085976520747, 'subsample': 0.7185686411012703, 'colsample_bytree': 0.9532584270470845}. Best is trial 33 with value: 0.4088990823534232.
[I 2024-02-22 19:22:35,590] Trial 49 finished with value: 0.4221945314672893 and parameters: {'learning_rate': 0.09019318924818226, 'n_estimators': 482, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.06850946805760431, 'subsample': 0.6539844139717307, 'colsample_bytree': 0.8603906193746189}. Best is trial 33 with value: 0.4088990823534232.
[I 2024-02-22 19:22:35,966] Trial 50 finished with value: 0.4202919514834127 and parameters: {'learning_rate': 0.2578089880363022, 'n_estimators': 443, 'max_depth': 5, 'min_child_weight': 1, 'gamma': 0.2182048611895403, 'subsample': 0.6665038798475211, 'colsample_bytree': 0.9105362469359624}. Best is trial 33 

[I 2024-02-22 19:22:44,525] Trial 72 finished with value: 0.4185062191069015 and parameters: {'learning_rate': 0.09341612985919354, 'n_estimators': 491, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.33657624077325976, 'subsample': 0.6367723112135862, 'colsample_bytree': 0.7936268211031658}. Best is trial 71 with value: 0.40733117560239523.
[I 2024-02-22 19:22:44,909] Trial 73 finished with value: 0.41582930597487106 and parameters: {'learning_rate': 0.16650038480952123, 'n_estimators': 475, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.3684536779099233, 'subsample': 0.6520761179300446, 'colsample_bytree': 0.8468741054716097}. Best is trial 71 with value: 0.40733117560239523.
[I 2024-02-22 19:22:45,305] Trial 74 finished with value: 0.4851997690400191 and parameters: {'learning_rate': 0.6952540884623983, 'n_estimators': 497, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.1921552586273863, 'subsample': 0.6846368762477013, 'colsample_bytree': 0.7388284274042834}. Best is trial 

[I 2024-02-22 19:22:53,015] Trial 96 finished with value: 0.4338201723844549 and parameters: {'learning_rate': 0.1749329990390855, 'n_estimators': 414, 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0.2103204349445511, 'subsample': 0.6453172622103142, 'colsample_bytree': 0.8112174605381031}. Best is trial 76 with value: 0.40507268337739627.
[I 2024-02-22 19:22:53,466] Trial 97 finished with value: 0.4113077002805635 and parameters: {'learning_rate': 0.02380852101249517, 'n_estimators': 484, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.18165351603137486, 'subsample': 0.6203113890249594, 'colsample_bytree': 0.8619269278364239}. Best is trial 76 with value: 0.40507268337739627.
[I 2024-02-22 19:22:53,924] Trial 98 finished with value: 0.4152276596598746 and parameters: {'learning_rate': 0.024876912794186034, 'n_estimators': 499, 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0.18485601699190127, 'subsample': 0.6101973158394144, 'colsample_bytree': 0.864378910203239}. Best is trial 

最佳超参数组合： {'learning_rate': 0.11697586719332753, 'n_estimators': 418, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 0.20766198261966232, 'subsample': 0.669756753096141, 'colsample_bytree': 0.8327503951995238}
训练集的均方根误差 (RMSE)： 0.2680568642957661
测试集的均方根误差 (RMSE)： 0.4702055947242495
训练集的决定系数 (R2)： 0.8905123564675393
测试集的决定系数 (R2)： 0.6735642599210259


In [5]:
def calculate_adjusted_r2(r_squared, n, p):
    return 1 - (1 - r_squared) * (n - 1) / (n - p - 1)

# 计算训练集和测试集的校正后R2
n_train = X_train_selected.shape[0]
p_train = X_train_selected.shape[1]
n_test = X_test_selected.shape[0]
p_test = X_test_selected.shape[1]

train_adjusted_r2 = calculate_adjusted_r2(train_r2, n_train, p_train)
test_adjusted_r2 = calculate_adjusted_r2(test_r2, n_test, p_test)

# 输出校正后的R2
print("训练集的校正后决定系数 (Adjusted R2)：", train_adjusted_r2)
print("测试集的校正后决定系数 (Adjusted R2)：", test_adjusted_r2)

训练集的校正后决定系数 (Adjusted R2)： 0.7867699089191637
测试集的校正后决定系数 (Adjusted R2)： 0.5271760960561329
